In [1]:
import pandas as pd
import numpy as np
import torch

In [2]:
torch.cuda.is_available()

True

In [3]:
movie_complete = pd.read_csv('../TMDB_movie_dataset_v11.csv')

In [4]:
movie_complete.head()

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,original_title,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,keywords
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,...,Inception,"Cobb, a skilled thief who commits corporate es...",83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili","rescue, mission, dream, airplane, paris, franc..."
1,157336,Interstellar,8.417,32571,Released,2014-11-05,701729206,169,False,/pbrkL804c8yAv3zBZR4QPEafpAR.jpg,...,Interstellar,The adventures of a group of explorers who mak...,140.241,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English,"rescue, future, spacecraft, race against time,..."
2,155,The Dark Knight,8.512,30619,Released,2008-07-16,1004558444,152,False,/nMKdUUepR0i5zn0y1T4CsSB5chy.jpg,...,The Dark Knight,Batman raises the stakes in his war on crime. ...,130.643,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","English, Mandarin","joker, sadism, chaos, secret identity, crime f..."
3,19995,Avatar,7.573,29815,Released,2009-12-15,2923706026,162,False,/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg,...,Avatar,"In the 22nd century, a paraplegic Marine is di...",79.932,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","English, Spanish","future, society, culture clash, space travel, ..."
4,24428,The Avengers,7.710,29166,Released,2012-04-25,1518815515,143,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,...,The Avengers,When an unexpected enemy emerges and threatens...,98.082,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,Some assembly required.,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"English, Hindi, Russian","new york city, superhero, shield, based on com..."


In [5]:
movie_complete.columns

Index(['id', 'title', 'vote_average', 'vote_count', 'status', 'release_date',
       'revenue', 'runtime', 'adult', 'backdrop_path', 'budget', 'homepage',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'tagline', 'genres',
       'production_companies', 'production_countries', 'spoken_languages',
       'keywords'],
      dtype='object')

In [6]:
# title, overview,
# adult, genres, keywords
np.random.seed(42)
movie = movie_complete.copy()
movie = movie[movie['adult'] == False]
movie = movie.sort_values(by='popularity', ascending=False)
movie = movie[['genres','keywords','title','overview']].dropna()
movie = movie[:20_000]
movie

,genres,keywords,title,overview
3870,"Action, Science Fiction, Adventure","armor, superhero, family relationships, family...",Blue Beetle,Recent college grad Jaime Reyes returns home f...
5051,"Action, Drama, Adventure","based on true story, racing, based on video ga...",Gran Turismo,The ultimate wish-fulfillment tale of a teenag...
7930,"Horror, Mystery, Thriller","france, bullying, sequel, religion, demon, got...",The Nun II,"In 1956 France, a priest is violently murdered..."
2129,"Action, Science Fiction, Horror","based on novel or book, sequel, shark, kaiju, ...",Meg 2: The Trench,An exploratory dive into the deepest depths of...
9970,"Action, Mystery, Thriller, Crime","assassination, bomb, remake, divorce, duringcr...",Retribution,When a mysterious caller puts a bomb under his...
...,...,...,...,...
25595,"Horror, Adventure, Thriller","river, camping, hiking, wilderness, vacation, ...",Rituals,Five doctors go camping in the remote woods of...
24331,"Action, Comedy, Western",brothel,The Cheyenne Social Club,"Two cowboys inherit a ""social club"" specializi..."
25204,Comedy,"man child, tuscany, italy, sex comedy",Lucignolo,"Red-haired, sex-obsessed manchild Lucio lazes ..."
9099,"Drama, Comedy",childhood sexual abuse,Little Tickles,Odette is a 8-yr-old girl who loves to dance a...


In [7]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict

c:\Users\danie\miniforge3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
def make_input_prompt(row):
    return (f"genres: {row['genres']}\n"
            f"keywords: {row['keywords']}\n"
            "Generate title and overview:")

def make_target_text(row):
    return (f"title: {row['title']}\n"
            f"overview: {row['overview']}")

movie['input_text'] = movie.apply(make_input_prompt, axis=1)
movie['target_text'] = movie.apply(make_target_text, axis=1)
movie.head()

,genres,keywords,title,overview,input_text,target_text
3870,"Action, Science Fiction, Adventure","armor, superhero, family relationships, family...",Blue Beetle,Recent college grad Jaime Reyes returns home f...,"genres: Action, Science Fiction, Adventure\nke...",title: Blue Beetle\noverview: Recent college g...
5051,"Action, Drama, Adventure","based on true story, racing, based on video ga...",Gran Turismo,The ultimate wish-fulfillment tale of a teenag...,"genres: Action, Drama, Adventure\nkeywords: ba...",title: Gran Turismo\noverview: The ultimate wi...
7930,"Horror, Mystery, Thriller","france, bullying, sequel, religion, demon, got...",The Nun II,"In 1956 France, a priest is violently murdered...","genres: Horror, Mystery, Thriller\nkeywords: f...","title: The Nun II\noverview: In 1956 France, a..."
2129,"Action, Science Fiction, Horror","based on novel or book, sequel, shark, kaiju, ...",Meg 2: The Trench,An exploratory dive into the deepest depths of...,"genres: Action, Science Fiction, Horror\nkeywo...",title: Meg 2: The Trench\noverview: An explora...
9970,"Action, Mystery, Thriller, Crime","assassination, bomb, remake, divorce, duringcr...",Retribution,When a mysterious caller puts a bomb under his...,"genres: Action, Mystery, Thriller, Crime\nkeyw...",title: Retribution\noverview: When a mysteriou...


In [9]:
# simple random split
train_df = movie.sample(frac=0.8, random_state=42)
val_df = movie.drop(train_df.index)

In [10]:
train_dataset = Dataset.from_pandas(train_df[['input_text','target_text']])
val_dataset = Dataset.from_pandas(val_df[['input_text','target_text']])

tv_dict = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset
})

In [11]:
model_name = "t5-base" 
tokenizer = T5Tokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    # we feed the model the input_text, and want to predict target_text
    inputs = tokenizer(examples["input_text"], padding="max_length", truncation=True, max_length=128)
    targets = tokenizer(examples["target_text"], padding="max_length", truncation=True, max_length=256)
    inputs["labels"] = targets["input_ids"]
    return inputs

tokenized_datasets = tv_dict.map(tokenize_function, batched=True)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Map: 100%|██████████| 4000/4000 [00:01<00:00, 2842.78 examples/s]


In [12]:
model = T5ForConditionalGeneration.from_pretrained(model_name)

training_args = TrainingArguments(
 output_dir="t5_movies_base_20000",
    eval_strategy="steps",    # epoch or steps
    logging_steps=500,             # log training metrics every 500 steps
    save_steps=1000,               # save model checkpoint every 1000 steps
    save_total_limit=2,  
             
    
    num_train_epochs=2,            
    
    # batch sizes 16 seems to be the max
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    
    learning_rate=3e-5,            
    weight_decay=0.01,             
    
    # only if GPU is available
    fp16=True,                     
    
    # Might wanna turn this on later to host online
    push_to_hub=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"]
)

In [13]:
trainer.train()
trainer.save_model("t5_movies_base_20000")
tokenizer.save_pretrained("t5_movies_base_20000")

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss
500,1.455900,0.986246
1000,1.044400,0.971760
1500,1.034200,0.965859
2000,1.026500,0.964045


('t5_movies_base_20000\\tokenizer_config.json',
 't5_movies_base_20000\\special_tokens_map.json',
 't5_movies_base_20000\\spiece.model',
 't5_movies_base_20000\\added_tokens.json')

In [41]:
tokenizer = T5Tokenizer.from_pretrained("t5_movies_base_20000")
model = T5ForConditionalGeneration.from_pretrained("t5_movies_base_20000")

def generate_tv_show_description(genres, keywords):
    # Create a prompt
    prompt = (f"genres: {genres}\n"
            f"keywords: {keywords}\n"
            "Generate title and overview:")

    # Tokenize
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    # Generate
    output_ids = model.generate(
        input_ids,
        max_length=256,
        do_sample=True,
        #top_p=0.95,       # nucleus/top-p sampling
        top_k=30, #bigger = more diverse. So far, k<=30 is good. k>50 sucks.
        temperature=1.0, #more temperature = more
        repetition_penalty=1.2
    )
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return output_text

test_genres = "Drama, Romance"
test_keywords = "Artificial Intelligence, Technology, Valentine's day"

result = generate_tv_show_description(test_genres, test_keywords)
print(result)

title: Valentine's Day overview: When two friends of their supposedly 'loved' mother, Dame Anita, start meeting each other for the first time at their college graduation party, they meet again at a school lunch party. They meet and soon become lovers, but the love of an older woman's life has already been rekindled.


In [48]:
test_genres = "Drama, Romance"
test_keywords = "Artificial Intelligence, Technology, Valentine's day"

result = generate_tv_show_description(test_genres, test_keywords)
print(result)

title: A Secret of Love overview: An intelligent woman starts a business with a robot that predicts her future.


In [33]:
movie['keywords'].value_counts()

keywords
softcore                                                                                                                                                                                                                                                                                                                                                                 282
woman director                                                                                                                                                                                                                                                                                                                                                            92
anime                                                                                                                                                                                                                                                                